In [1]:
#| hide
%load_ext autoreload
%autoreload 2

# Dune Dashboard Base Class

> Base class for pulling data from dune

In [2]:
#| default_exp dune.dune

In [3]:
#| export
import os
import dotenv
import pandas as pd
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import Query
import diskcache

In [4]:
# | export
class DuneWrapper:
    
    def __init__(self):
        dotenv.load_dotenv()
        self.dune = DuneClient(os.environ["DUNE_API_KEY"])
        self.cache = diskcache.Cache('cache')
        
    def refresh_into_dataframe(self, query_id, params=[], name="Sample Query", update_cache=False):
        params_formatted = []
        for parameter in params:          
            params_formatted.append(getattr(QueryParameter, parameter["type"] + "_type")(name=parameter["name"], value=parameter["value"]))
            
        query = Query(
            name=name,
            query_id=query_id,
            params=params_formatted
        )

        cache_key = f'refresh_into_dataframe-{query_id}'
        if (cache_key in self.cache) and not update_cache:
            df = self.cache[cache_key]
        else:
            df = self.dune.refresh_into_dataframe(query=query)
            self.cache[cache_key] = df
        return df


In [5]:
dune = DuneWrapper()

In [6]:
dune.refresh_into_dataframe(1898885)

,block_date,avg_mint_price,avg_burn_price,avg_mkt_price
0,2022-01-25T00:00:00Z,1.956345888613234,<nil>,2.3050899888800456
1,2022-01-26T00:00:00Z,2.893025644547676,<nil>,2.5920101807901506
2,2022-01-27T00:00:00Z,2.864333966225624,2.2216159017515933,2.5403510974065853
3,2022-01-28T00:00:00Z,2.8263603268647635,2.24996276540192,2.4776305971040147
4,2022-01-29T00:00:00Z,2.9437308337890533,<nil>,2.683241567655875
...,...,...,...,...
468,2023-05-09T00:00:00Z,0.5622850229364372,0.5622850229364372,0.5258280581358534
469,2023-05-10T00:00:00Z,0.5608069285035435,0.5608069285035435,0.5321566407096778
470,2023-05-11T00:00:00Z,0.5596423808136822,0.5596423808136822,0.5216465103908255
471,2023-05-12T00:00:00Z,0.5540953786399315,0.5540953786399315,0.4488854737838069


In [8]:
dune.refresh_into_dataframe(1752257, update_cache=True)

2023-06-26 20:21:23,589 INFO dune_client.base_client waiting for query execution 01H3XDXGJTAT41Y2QMW22KBFR8 to complete: ExecutionState.EXECUTING


,total_pool_value,reserve_pool_value,common_pool_value
0,1.026493e+06,418519.660202,607972.869308


In [12]:
dune.refresh_into_dataframe(1752257, update_cache=True, params=[{
    "type": "date",
    "name": "2. End Date",
    "value": "2055-05-04 00:00:00",
}])

2023-06-26 20:26:14,303 INFO dune_client.base_client waiting for query execution 01H3XE6CFAG1NKGAYQG00WA86P to complete: ExecutionState.EXECUTING


,total_pool_value,reserve_pool_value,common_pool_value
0,598970.662378,204683.097109,394287.56527


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()